In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.api import VAR

In [2]:
data = pd.read_excel('final data.xlsx',index_col=0)
data        # 데이터의 경우, 가격 상승 이후 후행하는 기사의 문제를 해결하고자 수입가격을 기사 발표 이후 1개월 뒤로 설정함.

,"Crude oil, Dubai","Coal, Australian","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",U$/MT,ChatGPT_Score,BERT_Score
Month,,,,,,,,
2013-10-01,106.302273,79.4125,3.6739,11.370000,15.300000,757.827849,-2,0
2013-11-01,105.853810,82.2500,3.6174,11.420000,15.400000,761.290646,-2,1
2013-12-01,107.919524,84.3375,4.2366,11.550000,16.380000,810.300120,-1,0
2014-01-01,104.010000,81.6100,4.7008,11.590000,16.670000,861.274592,-4,0
2014-02-01,104.940000,76.2900,5.9733,11.300000,16.760000,861.201677,-5,0
...,...,...,...,...,...,...,...,...
2022-03-01,113.110000,314.0400,4.8839,42.392971,15.111385,695.031958,-20,-2
2022-04-01,102.680000,310.8500,6.5306,32.204187,16.287524,723.310581,-14,-1
2022-05-01,108.320000,371.8600,8.1373,29.170723,16.679091,762.065675,-8,-1


In [3]:
def adf_test(df):     # 정상성 검정
  result = adfuller(df)
  print('ADF Statistics: %f' % result[0])
  print('p-value: %f' % result[1])

In [4]:
dubai = data['Crude oil, Dubai']
aus = data['Coal, Australian']
us = data['Natural gas, US']
europe = data['Natural gas, Europe']
japan = data['Liquefied natural gas, Japan']
mt = data['U$/MT']
gpt = data['ChatGPT_Score']
bert = data['BERT_Score']

In [5]:
print('ADF TEST : dubai')       # 차분하지 않았을 때 유의수준 5%에서 수입가격과 bert를 제외한 모든 데이터가 정상성을 만족하지 못함.
adf_test(dubai)
print('ADF TEST : aus')
adf_test(aus)
print('ADF TEST : us')
adf_test(us)
print('ADF TEST : europe')
adf_test(europe)
print('ADF TEST : japan')
adf_test(japan)
print('ADF TEST : mt')
adf_test(mt)
print('ADF TEST : gpt')
adf_test(gpt)
print('ADF TEST : bert')
adf_test(bert)

ADF TEST : dubai
ADF Statistics: -2.107554
p-value: 0.241488
ADF TEST : aus
ADF Statistics: 0.470649
p-value: 0.983953
ADF TEST : us
ADF Statistics: -1.375718
p-value: 0.593860
ADF TEST : europe
ADF Statistics: -1.751958
p-value: 0.404560
ADF TEST : japan
ADF Statistics: -1.930809
p-value: 0.317709
ADF TEST : mt
ADF Statistics: -2.936019
p-value: 0.041317
ADF TEST : gpt
ADF Statistics: -1.996013
p-value: 0.288324
ADF TEST : bert
ADF Statistics: -9.014582
p-value: 0.000000


In [6]:
final_data = data.diff().diff().dropna()        # 2차차분실시

In [7]:
dubai2 = final_data['Crude oil, Dubai']
aus2 = final_data['Coal, Australian']
us2 = final_data['Natural gas, US']
europe2 = final_data['Natural gas, Europe']
japan2 = final_data['Liquefied natural gas, Japan']
mt2 = final_data['U$/MT']
gpt2 = final_data['ChatGPT_Score']
bert2 = final_data['BERT_Score']

In [8]:
print('ADF TEST : dubai')              # 2차차분했을 때 모든 데이터가 정상성을 만족하지 못함.
adf_test(dubai2)
print('ADF TEST : aus')
adf_test(aus2)
print('ADF TEST : us')
adf_test(us2)
print('ADF TEST : europe')
adf_test(europe2)
print('ADF TEST : japan')
adf_test(japan2)
print('ADF TEST : mt')
adf_test(mt2)
print('ADF TEST : gpt')
adf_test(gpt2)
print('ADF TEST : bert')
adf_test(bert2)

ADF TEST : dubai
ADF Statistics: -5.844245
p-value: 0.000000
ADF TEST : aus
ADF Statistics: -6.692028
p-value: 0.000000
ADF TEST : us
ADF Statistics: -6.559052
p-value: 0.000000
ADF TEST : europe
ADF Statistics: -7.901918
p-value: 0.000000
ADF TEST : japan
ADF Statistics: -7.213086
p-value: 0.000000
ADF TEST : mt
ADF Statistics: -2.990598
p-value: 0.035782
ADF TEST : gpt
ADF Statistics: -5.547053
p-value: 0.000002
ADF TEST : bert
ADF Statistics: -7.139745
p-value: 0.000000


In [9]:
from statsmodels.tsa.stattools import grangercausalitytests
maxlag=12
test = 'ssr_chi2test'

In [10]:
def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    

    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(final_data, variables = final_data.columns)       # 그래인저 인과검정

c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(


c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  warnings.warn(
c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\stattools.py:1488: FutureWarning: verbose is deprecated since functions should not print results
  

,"Crude oil, Dubai_x","Coal, Australian_x","Natural gas, US_x","Natural gas, Europe_x","Liquefied natural gas, Japan_x",U$/MT_x,ChatGPT_Score_x,BERT_Score_x
"Crude oil, Dubai_y",1.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0255,0.0810
"Coal, Australian_y",0.3487,1.0000,0.0000,0.0000,0.0000,0.0000,0.0575,0.0000
"Natural gas, US_y",0.0029,0.0000,1.0000,0.0000,0.1454,0.0497,0.0000,0.0123
"Natural gas, Europe_y",0.0006,0.0000,0.0000,1.0000,0.0000,0.0000,0.1613,0.0000
"Liquefied natural gas, Japan_y",0.0000,0.0000,0.0008,0.0000,1.0000,0.0000,0.0090,0.0045
U$/MT_y,0.0000,0.0000,0.0000,0.0000,0.0002,1.0000,0.0010,0.0000
ChatGPT_Score_y,0.0257,0.0009,0.0023,0.0977,0.0014,0.0172,1.0000,0.0000
BERT_Score_y,0.2514,0.0001,0.0000,0.0000,0.0137,0.0000,0.1219,1.0000


In [11]:
max_lag = 6
model = VAR(final_data)
aic_values = []
bic_values = []

# Compute AIC and BIC for lags up to max_lag
for i in range(1, max_lag + 1):
    result = model.fit(i)
    aic_values.append(result.aic)
    bic_values.append(result.bic)

optimal_lag_aic = aic_values.index(min(aic_values)) + 1
optimal_lag_bic = bic_values.index(min(bic_values)) + 1

optimal_lag_aic, optimal_lag_bic

c:\Users\82105\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


(6, 2)

In [12]:
# Recompute HQ for lags up to max_lag
hq_values = []
for i in range(1, max_lag + 1):
    result = model.fit(i)
    hq_values.append(result.hqic)

optimal_lag_hq = hq_values.index(min(hq_values)) + 1

criteria = {
    'AIC': optimal_lag_aic,
    'BIC': optimal_lag_bic,
    'HQ': optimal_lag_hq,
}

criteria        # 최적시차 결정

{'AIC': 6, 'BIC': 2, 'HQ': 3}

In [13]:
# Prepare a dataframe to display the values for each lag
criteria_df = pd.DataFrame({
    'Lag': range(1, max_lag + 1),
    'AIC': aic_values,
    'BIC': bic_values,
    'HQ': hq_values,
})

criteria_df

,Lag,AIC,BIC,HQ
0,1,24.053965,25.895717,24.799938
1,2,20.837043,24.337007,22.254298
2,3,19.886898,25.065354,21.983286
3,4,19.296032,26.173682,22.079541
4,5,18.571904,27.169877,22.050658
5,6,17.978263,28.318133,22.160527
